# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察填補缺值以及 標準化 / 最小最大化 對數值的影響

# [作業重點]
- 觀察替換不同補缺方式, 對於特徵的影響 (In[4]~In[6], Out[4]~Out[6])
- 觀察替換不同特徵縮放方式, 對於特徵的影響 (In[7]~In[8], Out[7]~Out[8])

In [13]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = 'data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['survived']
ids = df_test['passenger_id']
df_train = df_train.drop(['passenger_id', 'survived'] , axis=1)
df_test = df_test.drop(['passenger_id'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,3,"Smyth, Miss. Julia",female,NaN,0,0,335432,7.7333,NaN,Q,13,NaN,NaN
1,3,"Cacic, Mr. Luka",male,38.0,0,0,315089,8.6625,NaN,S,NaN,NaN,Croatia
2,3,"Van Impe, Mrs. Jean Baptiste (Rosalie Paula Go...",female,30.0,1,1,345773,24.1500,NaN,S,NaN,NaN,NaN
3,2,"Hocking, Mrs. Elizabeth (Eliza Needs)",female,54.0,1,3,29105,23.0000,NaN,S,4,NaN,"Cornwall / Akron, OH"
4,2,"Veal, Mr. James",male,40.0,0,0,28221,13.0000,NaN,S,NaN,NaN,"Barre, Co Washington, VT"


In [14]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

6 Numeric Features : ['pclass', 'age', 'sibsp', 'parch', 'fare', 'body']



In [15]:
# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
train_num = train_Y.shape[0]
df.head()

,pclass,age,sibsp,parch,fare,body
0,3,NaN,0,0,7.7333,NaN
1,3,38.0,0,0,8.6625,NaN
2,3,30.0,1,1,24.1500,NaN
3,2,54.0,1,3,23.0000,NaN
4,2,40.0,0,0,13.0000,NaN


# 作業1
* 試著在補空值區塊, 替換並執行兩種以上填補的缺值, 看看何者比較好?

In [22]:
# 空值分別補 0,1 ; 做羅吉斯迴歸

for value in [0,1]:
    df_m1 = df.fillna(value)
    train_X = df_m1[:train_num]
    estimator = LogisticRegression()
    acc = cross_val_score(estimator, train_X, train_Y, cv=5).mean()
    print(f'fill value: {value} , acc = {acc}' )

fill value: 0 , acc = 0.7164705882352941
fill value: 1 , acc = 0.7164705882352941


# 作業2
* 使用不同的標準化方式 ( 原值 / 最小最大化 / 標準化 )，搭配羅吉斯迴歸模型，何者效果最好?

In [23]:
df_tmp = df.fillna(0)
df_tmp = MinMaxScaler().fit_transform(df_tmp)
train_X = df_tmp[:train_num]
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.7105882352941176